In [1]:
import zipfile
import xml.etree.ElementTree as ET
from docx import Document
import docx.oxml.table
import docx.oxml.text.paragraph
import docx.table
import docx.text.paragraph
from docx.oxml.ns import qn
import xml.etree.ElementTree as ET
import lxml.etree as LET
from xml.etree.ElementTree import ElementTree
from docx import Document
from docx.opc.constants import RELATIONSHIP_TYPE as RT
from docx.shared import Length

## データの読み込み

In [2]:
docx_path = 'サービス提供契約書_2024年4月1日_バージョン1.docx'

In [3]:
doc = docx.Document(docx_path)

In [4]:
len(doc.paragraphs)

57

In [5]:
# .docxファイルからnumbering.xmlを抽出

def extract_numbering_xml(docx_path):
    # zipファイルを読み込む
    with zipfile.ZipFile(docx_path, 'r') as docx_zip:
        if 'word/numbering.xml' in docx_zip.namelist():
            return docx_zip.read('word/numbering.xml')
    return None

In [6]:
# xmlの取得
xml_content = extract_numbering_xml(docx_path)
xml_content

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\r\n<w:numbering xmlns:wpc="http://schemas.microsoft.com/office/word/2010/wordprocessingCanvas" xmlns:cx="http://schemas.microsoft.com/office/drawing/2014/chartex" xmlns:cx1="http://schemas.microsoft.com/office/drawing/2015/9/8/chartex" xmlns:cx2="http://schemas.microsoft.com/office/drawing/2015/10/21/chartex" xmlns:cx3="http://schemas.microsoft.com/office/drawing/2016/5/9/chartex" xmlns:cx4="http://schemas.microsoft.com/office/drawing/2016/5/10/chartex" xmlns:cx5="http://schemas.microsoft.com/office/drawing/2016/5/11/chartex" xmlns:cx6="http://schemas.microsoft.com/office/drawing/2016/5/12/chartex" xmlns:cx7="http://schemas.microsoft.com/office/drawing/2016/5/13/chartex" xmlns:cx8="http://schemas.microsoft.com/office/drawing/2016/5/14/chartex" xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006" xmlns:aink="http://schemas.microsoft.com/office/drawing/2016/ink" xmlns:am3d="http://schemas.microsoft.com/office/dra

In [7]:
WORD_NAMESPACE = 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'
ns = {'w': WORD_NAMESPACE}

In [8]:
# 番号付け定義の読み込みと解析関数
def load_numbering_definitions(docx_path):
    with zipfile.ZipFile(docx_path) as docx_zip:
        with docx_zip.open('word/numbering.xml') as numbering_xml:
            xml_content = numbering_xml.read()
    numbering_root = ET.fromstring(xml_content)
    
    numId_abstractNumId_map = {}
    abstractNumId_numFmt_map = {}
    
    for num in numbering_root.findall(f'w:num', ns):
        numId = num.get(f'{{{WORD_NAMESPACE}}}numId')
        abstractNumId = num.find('w:abstractNumId', ns).get(f'{{{WORD_NAMESPACE}}}val')
        numId_abstractNumId_map[numId] = abstractNumId
    
    for abstractNum in numbering_root.findall(f'w:abstractNum', ns):
        abstractNumId = abstractNum.get(f'{{{WORD_NAMESPACE}}}abstractNumId')
        numFmt = abstractNum.find(f'.//w:numFmt', ns).get(f'{{{WORD_NAMESPACE}}}val')
        abstractNumId_numFmt_map[abstractNumId] = numFmt

    return numId_abstractNumId_map, abstractNumId_numFmt_map

In [9]:
# 段落の詳細抽出関数
def extract_paragraph_details(docx_path, numId_abstractNumId_map, abstractNumId_numFmt_map):
    doc = Document(docx_path)
    paragraph_details = []
    
    for para in doc.paragraphs:
        pPr_elem = para._element.find('.//w:pPr', ns)
        numId_elem = pPr_elem.find('.//w:numId', ns) if pPr_elem is not None else None
        ilvl_elem = pPr_elem.find('.//w:ilvl', ns) if pPr_elem is not None else None
        
        # 要素が存在するかどうかのチェックを修正
        if numId_elem is not None and ilvl_elem is not None:
            numId = numId_elem.get(f'{{{WORD_NAMESPACE}}}val')
            ilvl = ilvl_elem.get(f'{{{WORD_NAMESPACE}}}val')
            abstractNumId = numId_abstractNumId_map.get(numId, 'Unknown')
            numFmt = abstractNumId_numFmt_map.get(abstractNumId, 'Unknown')
            pPr_info = 'Exists' if ilvl_elem is not None else 'Not Found'
            paragraph_details.append((para.text, numId, ilvl, numFmt, pPr_info))
    
    return paragraph_details

In [10]:
numId_abstractNumId_map, abstractNumId_numFmt_map = load_numbering_definitions(docx_path)
paragraph_details = extract_paragraph_details(docx_path, numId_abstractNumId_map, abstractNumId_numFmt_map)

for detail in paragraph_details:
    print(f"Text: {detail[0]}, NumId: {detail[1]}, Level: {detail[2]}, Format: {detail[3]}, pPr: {detail[4]}")

Text: 第1条（契約の目的）, NumId: 5, Level: 0, Format: upperRoman, pPr: Exists
Text: この契約は、甲方が乙方に対して[具体的なサービスや製品の名称]の提供に関する条件を定めるものである。, NumId: 5, Level: 1, Format: upperRoman, pPr: Exists
Text: 第2条（契約期間）, NumId: 5, Level: 0, Format: upperRoman, pPr: Exists
Text: この契約の有効期間は、2024年4月1日から2025年3月31日までとする。, NumId: 5, Level: 1, Format: upperRoman, pPr: Exists
Text: 第3条（サービスの内容）, NumId: 5, Level: 0, Format: upperRoman, pPr: Exists
Text: 甲方は乙方に対して、次のサービスを提供するものとする。, NumId: 5, Level: 1, Format: upperRoman, pPr: Exists
Text: [サービス内容1], NumId: 5, Level: 2, Format: upperRoman, pPr: Exists
Text: [サービス内容2], NumId: 5, Level: 2, Format: upperRoman, pPr: Exists
Text: 第4条（報酬）, NumId: 5, Level: 0, Format: upperRoman, pPr: Exists
Text: 乙方は、甲方が提供するサービスの対価として、[金額]を支払うものとする。, NumId: 5, Level: 1, Format: upperRoman, pPr: Exists
Text: 支払い条件、支払い方法については、[具体的な条件や方法]に従う。, NumId: 5, Level: 1, Format: upperRoman, pPr: Exists
Text: 第5条（契約の変更と解除）, NumId: 5, Level: 0, Format: upperRoman, pPr: Exists
Text: 本契約に関する変更や追加は、両当事者の書面に